## Imports

In [ ]:
import cv2 as cv
import os 
from matplotlib import pyplot as plt
import time
# import mediapipe as mp
import numpy as np

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.

## Getting camera to work

In [ ]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    # gray = cv.cvtColor(frame, cv.COLOR_RGB2RGBA)
    # Display the resulting frame
    cv.imshow('Video Feed', frame)
    if cv.waitKey(1) == ord('q'):
        break
 
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()